# Tensorflow basics

Creating a graph

In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [2]:
sess = tf.Session()

sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)


42


In [3]:
sess.close()

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [5]:
init = tf.global_variables_initializer() #prepare an init node, not run
                                         #right now
    
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [6]:
x1 = tf.Variable(1) #Creating a node adds it to the default graph
x1.graph is tf.get_default_graph()

True

What if you want to work with multiple graphs?

In [7]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [8]:
x2.graph is tf.get_default_graph()

False

In [9]:
tf.reset_default_graph()

In [10]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
    
#What happened to initializing w?? W isn't a global variable, it's a constant

10
15


In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with TensorFlow

In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing.keys()
housing.data[0]

array([   8.3252    ,   41.        ,    6.98412698,    1.02380952,
        322.        ,    2.55555556,   37.88      , -122.23      ])

In [13]:
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

XT = tf.transpose(X) #Transpose in preparation for normal equation

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()

## Batch Gradient Descent Linear Regression

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(housing_data_plus_bias)

from sklearn.preprocessing import Imputer

imputer = Imputer()
X_ready = imputer.fit_transform(X_scaled)

Now the data is ready for Batch Gradient Descent

In [15]:
n_epochs = 1000 #Number of times to loop through entire training set
learning_rate = 0.01

X = tf.constant(X_ready, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X,theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#mse just for keeping track of progress

#Manually computing gradient
##gradients = 2/m * tf.matmul(tf.transpose(X),error)

#Using Tensorflow autodiff
gradients = tf.gradients(mse,[theta])[0]

training_op = tf.assign(theta, theta - (learning_rate*gradients))
                        
init = tf.global_variables_initializer()
                        
with tf.Session() as sess:
    sess.run(init) #intiailize global variables
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
                        
    best_theta = theta.eval()
    print(best_theta)

Epoch  0 MSE =  7.9677873
Epoch  100 MSE =  4.9308724
Epoch  200 MSE =  4.854903
Epoch  300 MSE =  4.8401456
Epoch  400 MSE =  4.8313346
Epoch  500 MSE =  4.82485
Epoch  600 MSE =  4.819957
Epoch  700 MSE =  4.8162403
Epoch  800 MSE =  4.8134027
Epoch  900 MSE =  4.8112264
[[ 0.47845578]
 [ 0.873636  ]
 [ 0.14556561]
 [-0.31397176]
 [ 0.33048737]
 [ 0.00467944]
 [-0.04253284]
 [-0.6630776 ]
 [-0.6371763 ]]


In [16]:
#Same thing but with Tensorflow Optimizer

n_epochs = 1000 #Number of times to loop through entire training set
learning_rate = 0.01

X = tf.constant(X_ready, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X,theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#mse just for keeping track of progress

#Manually computing gradient
##gradients = 2/m * tf.matmul(tf.transpose(X),error)

#Using Tensorflow autodiff
##gradients = tf.gradients(mse,[theta])[0]

##training_op = tf.assign(theta, theta - (learning_rate*gradients))
  
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                      momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
                        
with tf.Session() as sess:
    sess.run(init) #intiailize global variables
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
                        
    best_theta = theta.eval()
    print(best_theta)

Epoch  0 MSE =  8.332202
Epoch  100 MSE =  4.805535
Epoch  200 MSE =  4.803355
Epoch  300 MSE =  4.8032637
Epoch  400 MSE =  4.803255
Epoch  500 MSE =  4.8032546
Epoch  600 MSE =  4.8032537
Epoch  700 MSE =  4.8032537
Epoch  800 MSE =  4.803254
Epoch  900 MSE =  4.803254
[[ 0.7720852 ]
 [ 0.8296224 ]
 [ 0.11875223]
 [-0.2655327 ]
 [ 0.30570108]
 [-0.00450284]
 [-0.03932639]
 [-0.8998784 ]
 [-0.8705341 ]]


## Using Placeholders to Minibatch GD

In [17]:
A = tf.placeholder(dtype=tf.float32, shape=(None,3)) # None means any size

B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    #Feed dictionary needs to be same shape as placeholder.
    #Double list [[]] means it's shape (1,3)
    #Using a single list [] fails because it considers it as shape (3,)
    
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
    
print(B_val_1)

print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [38]:
#Same thing but with Placeholders now

n_epochs = 1000 #Number of times to loop through entire training set
learning_rate = 0.01

X_ph = tf.placeholder(dtype="float", shape=(None,n+1))
y_ph = tf.placeholder(dtype="float", shape=(None,1))
batch_size = 100

X_ready[:,0] = 1
y_ready = housing.target.reshape(-1,1)

def fetch_batch(X, y, batch_size):
    #From X_ready, and y_ready, randomly select instances to create
    #A batch.
    
    random_idx = np.random.choice(X.shape[0], batch_size)
    
    X_batch = X[random_idx, :]
    y_batch = y[random_idx, :]
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X_ph,theta, name="predictions")
error = y_pred - y_ph
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                      momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

b,s = fetch_batch(X_ready,y_ready,batch_size)
print(type(s[0][0]))

with tf.Session() as sess:
    sess.run(init) #intiailize global variables
    for epoch in range(n_epochs):
        #if epoch % 100 ==0:
        #   print("Epoch ", epoch, "MSE = ", mse.eval())
            
        xbatch, ybatch = fetch_batch(X_ready, y_ready, batch_size)
        sess.run(training_op, feed_dict={X_ph:xbatch.astype(np.float32), 
                                         y_ph:ybatch.astype(np.float32)
                                        })
                        
    best_theta = theta.eval()
    print(best_theta)

<class 'numpy.float64'>
[[-3.06055450e+06]
 [-3.87086975e+06]
 [-1.04485638e+06]
 [ 8.14620050e+06]
 [-5.96727550e+06]
 [ 2.85651025e+06]
 [-2.24665568e+08]
 [-1.02349244e+06]
 [ 1.21915362e+06]]


## Saving and Restoring Models

In [42]:
#Same thing but Saves the models now

n_epochs = 1000 #Number of times to loop through entire training set
learning_rate = 0.01

X_ph = tf.placeholder(dtype="float", shape=(None,n+1))
y_ph = tf.placeholder(dtype="float", shape=(None,1))
batch_size = 100

X_ready[:,0] = 1
y_ready = housing.target.reshape(-1,1)

def fetch_batch(X, y, batch_size):
    #From X_ready, and y_ready, randomly select instances to create
    #A batch.
    
    random_idx = np.random.choice(X.shape[0], batch_size)
    
    X_batch = X[random_idx, :]
    y_batch = y[random_idx, :]
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X_ph,theta, name="predictions")
error = y_pred - y_ph
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                      momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init) #intiailize global variables
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            save_path = saver.save(sess,"./tmp/my_model.ckpt")
            
        xbatch, ybatch = fetch_batch(X_ready, y_ready, batch_size)
        sess.run(training_op, feed_dict={X_ph:xbatch.astype(np.float32), 
                                         y_ph:ybatch.astype(np.float32)
                                        })
    best_theta = theta.eval()
    print(best_theta)
    save_path = saver.save(sess,"./tmp/my_model_final.ckpt")

[[ 2.3835254 ]
 [ 0.7338193 ]
 [ 0.03696795]
 [ 0.26960617]
 [-0.06930675]
 [-0.5359972 ]
 [21.232433  ]
 [-0.87486464]
 [-0.96384573]]
